In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from model_mo2cap2 import mo2cap_ours
from tqdm import tqdm
import os

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = TempModel(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		image = test_batch['image'].cuda()
		image_zoom = test_batch['image_zoom'].cuda()
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {
			'image':image,
			# 'image_zoom' : image_zoom,
			}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp808'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp808/0.pth',
 '/workspace/2d_to_3d/apps/exp808/1.pth',
 '/workspace/2d_to_3d/apps/exp808/2.pth',
 '/workspace/2d_to_3d/apps/exp808/3.pth',
 '/workspace/2d_to_3d/apps/exp808/4.pth',
 '/workspace/2d_to_3d/apps/exp808/5.pth',
 '/workspace/2d_to_3d/apps/exp808/6.pth',
 '/workspace/2d_to_3d/apps/exp808/7.pth',
 '/workspace/2d_to_3d/apps/exp808/8.pth',
 '/workspace/2d_to_3d/apps/exp808/9.pth',
 '/workspace/2d_to_3d/apps/exp808/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	_path = f'/workspace/2d_to_3d/apps/eval/ours_depth_sil_sep'
	_name = f'baseline808_{i}.json'
	if not os.path.exists(_path):
		os.makedirs(_path)

	main(model_path=p,output_path=os.path.join(_path,_name))

07:55:49 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 153.8112335205078, 'LowerStretching': 162.11814880371094, 'Gaming': 156.08302307128906, 'UpperStretching': 155.8325958251953, 'Gesticuling': 127.20086669921875, 'Talking': 121.30593872070312, 'Reacting': 123.92985534667969, 'Walking': 154.49005126953125, 'Greeting': 103.63054656982422, 'Patting': 137.07467651367188}, 'UpperBody': {'All': 123.27918243408203, 'LowerStretching': 101.4001693725586, 'Gaming': 109.60456848144531, 'UpperStretching': 133.88357543945312, 'Gesticuling': 97.03599548339844, 'Talking': 115.64032745361328, 'Reacting': 115.31871032714844, 'Walking': 117.22120666503906, 'Greeting': 92.07872009277344, 'Patting': 111.30687713623047}, 'LowerBody': {'All': 184.34327697753906, 'LowerStretching': 222.83612060546875, 'Gaming': 202.56149291992188, 'UpperStretching': 177.78160095214844, 'Gesticuling': 157.36572265625, 'Talking': 126.9715347290039, 'Reacting': 132.54100036621094, 'Walking': 191.75888061523438, 'Greeting': 115.18238830566406, 'Patting': 162.

  7%|▋         | 401/5352 [01:04<10:46,  7.65it/s]

{'FullBody': {'All': 153.4425506591797, 'LowerStretching': 163.1786346435547, 'Gaming': 149.3035430908203, 'UpperStretching': 154.971923828125, 'Gesticuling': 124.08272552490234, 'Talking': 119.62943267822266, 'Reacting': 124.24382019042969, 'Walking': 154.2637176513672, 'Greeting': 107.25211334228516, 'Patting': 146.88523864746094}, 'UpperBody': {'All': 123.47708129882812, 'LowerStretching': 101.5694351196289, 'Gaming': 114.25054931640625, 'UpperStretching': 134.20791625976562, 'Gesticuling': 96.6701889038086, 'Talking': 116.86051940917969, 'Reacting': 112.48426055908203, 'Walking': 116.03961944580078, 'Greeting': 99.15692138671875, 'Patting': 122.14008331298828}, 'LowerBody': {'All': 183.4080047607422, 'LowerStretching': 224.78785705566406, 'Gaming': 184.35650634765625, 'UpperStretching': 175.73593139648438, 'Gesticuling': 151.49526977539062, 'Talking': 122.39836120605469, 'Reacting': 136.0033416748047, 'Walking': 192.48780822753906, 'Greeting': 115.3473129272461, 'Patting': 171.6303

 11%|█         | 601/5352 [01:32<11:17,  7.01it/s]

...

 93%|█████████▎| 5001/5352 [11:47<00:52,  6.65it/s]

{'FullBody': {'All': 77.5367660522461, 'LowerStretching': 76.79638671875, 'Walking': 76.80391693115234, 'UpperStretching': 79.62408447265625, 'Reacting': 72.35968780517578, 'Gesticuling': 63.15317153930664, 'Patting': 74.08047485351562, 'Greeting': 62.70763397216797, 'Talking': 59.67443084716797, 'Gaming': 74.63856506347656}, 'UpperBody': {'All': 56.92799377441406, 'LowerStretching': 47.99187469482422, 'Walking': 50.50464630126953, 'UpperStretching': 61.194236755371094, 'Reacting': 58.72312545776367, 'Gesticuling': 55.77388000488281, 'Patting': 54.92124938964844, 'Greeting': 50.633731842041016, 'Talking': 49.6760368347168, 'Gaming': 56.00310134887695}, 'LowerBody': {'All': 98.14553833007812, 'LowerStretching': 105.60089874267578, 'Walking': 103.10318756103516, 'UpperStretching': 98.05392456054688, 'Reacting': 85.9962387084961, 'Gesticuling': 70.53245544433594, 'Patting': 93.23969268798828, 'Greeting': 74.78153991699219, 'Talking': 69.67284393310547, 'Gaming': 93.2740249633789}}


 97%|█████████▋| 5201/5352 [12:15<00:25,  6.00it/s]

{'FullBody': {'All': 77.52889251708984, 'LowerStretching': 76.76700592041016, 'Walking': 76.71990966796875, 'UpperStretching': 79.63835906982422, 'Reacting': 72.29570007324219, 'Gesticuling': 62.96498489379883, 'Patting': 74.24246215820312, 'Greeting': 62.9105339050293, 'Talking': 59.794673919677734, 'Gaming': 74.64703369140625}, 'UpperBody': {'All': 56.934608459472656, 'LowerStretching': 48.00924301147461, 'Walking': 50.472877502441406, 'UpperStretching': 61.202728271484375, 'Reacting': 58.73481750488281, 'Gesticuling': 55.578853607177734, 'Patting': 54.9730110168457, 'Greeting': 50.72260284423828, 'Talking': 49.883338928222656, 'Gaming': 56.034698486328125}, 'LowerBody': {'All': 98.12319946289062, 'LowerStretching': 105.52477264404297, 'Walking': 102.96695709228516, 'UpperStretching': 98.07398223876953, 'Reacting': 85.85657501220703, 'Gesticuling': 70.35111236572266, 'Patting': 93.51192474365234, 'Greeting': 75.09848022460938, 'Talking': 69.70600128173828, 'Gaming': 93.25936126708984

100%|██████████| 5352/5352 [12:36<00:00,  7.07it/s]
10:14:56 [INFO] Main: Saving evaluation results...
10:14:56 [INFO] Main: Done.
